In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/20 00:02:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
df_green.registerTempTable('green')

/home/yic/Documents/Library/spark-3.3.1-bin-hadoop3/python/lib/pyspark.zip/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.


In [4]:
df_green.columns

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [5]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    
    SUM(tolls_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [6]:
df_green_revenue.show()

+-------------------+----+------+--------------+
|               hour|zone|amount|number_records|
+-------------------+----+------+--------------+
|2021-06-21 10:00:00|  55|  6.55|             3|
|2021-06-22 17:00:00| 166|   0.0|            13|
|2021-06-17 11:00:00| 244|   3.0|             4|
|2021-06-15 11:00:00|  51| 12.55|             3|
|2021-06-19 04:00:00| 250|   0.0|             1|
|2021-06-05 11:00:00| 166|   0.0|             4|
|2021-06-02 21:00:00|  35|   0.0|             2|
|2021-06-22 17:00:00|  95|   0.0|             8|
|2021-06-27 21:00:00|  74|   0.0|            12|
|2021-06-15 16:00:00| 197|  39.3|            16|
|2021-06-04 13:00:00| 181|   0.0|             3|
|2021-06-01 08:00:00| 183|   0.0|             1|
|2021-06-06 19:00:00| 226|   0.0|             1|
|2021-06-27 20:00:00| 256|  6.55|             1|
|2021-06-11 18:00:00|  75|   0.0|            29|
|2021-06-29 13:00:00|  42|  6.55|            10|
|2021-06-14 11:00:00| 166|   0.0|             3|
|2021-06-22 17:00:00

In [7]:
df_green_revenue.write.parquet('data/report/revenue/green', mode='overwrite')

In [8]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [9]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    SUM(tolls_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [10]:
df_yellow_revenue.write.parquet('data/report/revenue/yellow', mode='overwrite')

23/03/20 00:03:00 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


---

read green/yellow files

In [21]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')

In [22]:
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

rename the columns to distinguish green/yellow

In [23]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

In [24]:
df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [25]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [26]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

23/03/20 00:25:14 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


---

read green/yellow join file

In [27]:
df_join = spark.read.parquet('data/report/revenue/total')

join with zones

In [29]:
df_zones = spark.read.parquet('zones')

In [30]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [33]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')

23/03/20 00:37:27 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [31]:
df_result.show()

+-------------------+----+------------+--------------------+-------------+---------------------+----------+---------+--------------------+------------+
|               hour|zone|green_amount|green_number_records|yellow_amount|yellow_number_records|LocationID|  Borough|                Zone|service_zone|
+-------------------+----+------------+--------------------+-------------+---------------------+----------+---------+--------------------+------------+
|2020-12-31 18:00:00| 142|        null|                null|          0.0|                    2|       142|Manhattan| Lincoln Square East| Yellow Zone|
|2020-12-31 23:00:00|  43|        null|                null|          2.8|                    1|        43|Manhattan|        Central Park| Yellow Zone|
|2020-12-31 23:00:00| 137|        null|                null|          0.0|                    1|       137|Manhattan|            Kips Bay| Yellow Zone|
|2021-01-01 00:00:00|  13|        null|                null|          0.0|              